# ROS Integration


## Goals

* Run a local VLM as a service node using ROS2 Python RCL

## References

* [Understanding Services](https://docs.ros.org/en/kilted/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Services/Understanding-ROS2-Services.html)

## Setup

This notebook demonstrates integrating a Vision Language Model (VLM) with ROS2 using llama.cpp as the inference backend.

### Prerequisites

1. **llama.cpp server** running with a VLM model (e.g., LLaVA, BakLLaVA)
2. **ROS2** environment sourced
3. **vlm_ros package** built and sourced

### Architecture

- **llama.cpp server**: Runs the VLM model and exposes HTTP API (port 8080)
- **vlm_service node**: ROS2 node that subscribes to camera images and queries the VLM
- **Test script**: Python script to publish test images and receive VLM responses

## Start llama.cpp Server

**In a separate terminal**, start the llama.cpp server with a VLM model:

```bash
# Example: Download and run a VLM model
export PATH=/ryzers/llamacpp/build/bin/:$PATH
llama-server -hf ggml-org/SmolVLM-500M-Instruct-GGUF \
  --host 0.0.0.0 \
  --port 8080
```

Wait until you see "HTTP server listening" before proceeding.

## Try OpenAI API server directly

In [1]:
import requests
import base64
import cv2

# Load and encode an image
img = cv2.imread('/ryzers/notebooks/toucan.jpg')
_, buffer = cv2.imencode('.jpg', img)
img_base64 = base64.b64encode(buffer).decode('utf-8')

# Test the llama.cpp server
payload = {
  "prompt": "Describe what you see in this image.",
  "image_data": [{"data": img_base64, "id": 1}],
  "n_predict": 128
}

response = requests.post(
  'http://localhost:8080/completion',
  json=payload,
  timeout=30
)

print(f"Status: {response.status_code}")
print(f"Response: {response.json()['content']}")


Status: 200
Response: 

In the center of the image is a dark blue oval. Inside the oval is a light blue circle with a dark blue semi-transparent triangle. Inside the triangle is a light blue semi-transparent circle with a dark blue semi-transparent triangle.

The image is not realistic. There is no light blue circle in the image. The objects are not made of material. There is no background.

The image is a representation of an abstract thought. The shape of the oval is reminiscent of a mathematical equation, such as the Pythagorean theorem. The triangle and circle are geometric shapes, and the light blue circle


## Build the ROS Package

In [2]:
%%bash
source /opt/ros/kilted/setup.bash

# Build the vlm_ros package
cd /ryzers/notebooks/vlm_ros
colcon build --symlink-install

Starting >>> vlm_ros
Finished <<< vlm_ros [0.60s]

Summary: 1 package finished [0.67s]


## Start the VLM Service Node

Run the VLM service node in the background:

```bash
source /opt/ros/kilted/setup.bash
source /ryzers/notebooks/vlm_ros/install/setup.bash
ros2 run vlm_ros vlm_service
```

You should see this output:

```
[INFO] [1760575146.024196594] [vlm_service]: VLM Service started, connecting to http://localhost:8080
```

## Test with a Sample Image

Create a test script to publish an image and receive VLM responses:

In [3]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from std_msgs.msg import String
from cv_bridge import CvBridge
import cv2
import numpy as np

class VLMTester(Node):
    def __init__(self):
        super().__init__('vlm_tester')
        self.image_pub = self.create_publisher(Image, 'camera/image', 10)
        self.response_sub = self.create_subscription(String, 'vlm/response', self.response_callback, 10)
        self.bridge = CvBridge()
        self.latest_response = None
        
    def response_callback(self, msg):
        self.latest_response = msg.data
        print(f"\nVLM Response: {msg.data}\n")
        
    def publish_test_image(self, image_path):
        # Load and publish image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not load image: {image_path}")
            return
            
        msg = self.bridge.cv2_to_imgmsg(img, encoding='bgr8')
        self.image_pub.publish(msg)
        print(f"Published image: {image_path}")

# Initialize ROS
rclpy.init()
tester = VLMTester()
print("VLM tester node initialized")

VLM tester node initialized


### Publish image

In [5]:
# Publish a test image
tester.publish_test_image('/ryzers/notebooks/toucan.jpg')

# Spin to receive responses
import time
for _ in range(10):  # Wait up to 5 seconds for response
    rclpy.spin_once(tester, timeout_sec=0.5)
    if tester.latest_response:
        break

Published image: /ryzers/notebooks/toucan.jpg

VLM Response:  I can give you feedback and suggestions to improve the writing.





---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT